In [2]:
import numpy as np
import pyedflib
from matplotlib import pyplot as plt
from nitime import utils
from nitime import algorithms as alg
from nitime.timeseries import TimeSeries
from nitime.viz import plot_tseries
import csv
import pywt
import scipy.stats as sp
from sklearn.lda import LDA
from scipy import signal
from spectrum import *
from os import listdir
from os.path import isfile, join
from wyrm import processing as proc
from wyrm.types import Data
from wyrm.io import convert_mushu_data
from sklearn import metrics
from wyrm.processing import calculate_csp,segment_dat,apply_csp,append_epo
from wyrm.processing import select_channels
from wyrm.processing import swapaxes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.preprocessing import normalize

C:\Users\BCI-HCI-LAB-1\Anaconda2\lib\site-packages\sklearn\lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


In [3]:
files  = [f for f in listdir('Training data')]


channels = ['Fp1', 'AFp1', 'Fpz', 'AFp2', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8', 'FAF5', 'FAF1', 'FAF2', 'FAF6', 
                'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FFC7', 'FFC5', 'FFC3', 'FFC1', 'FFC2', 'FFC4', 
                'FFC6', 'FFC8', 'FT9', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'FT10', 'CFC7', 
                'CFC5', 'CFC3', 'CFC1', 'CFC2', 'CFC4', 'CFC6', 'CFC8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6'
                , 'T8', 'CCP7', 'CCP5', 'CCP3', 'CCP1', 'CCP2', 'CCP4', 'CCP6', 'CCP8', 'TP9', 'TP7', 'CP5', 'CP3', 'CP1',
                'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'TP10', 'PCP7', 'PCP5', 'PCP3', 'PCP1', 'PCP2', 'PCP4', 'PCP6', 'PCP8', 
                'P9', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10', 'PPO7', 'PPO5', 'PPO1', 'PPO2', 'PPO6',
                'PPO8', 'PO7', 'PO3', 'PO1', 'POz', 'PO2', 'PO4', 'PO8', 'OPO1', 'OPO2', 'O1', 'Oz', 'O2', 'OI1', 'OI2', 
                'I1', 'I2']

In [4]:
#for 1st subject

training_data = 'Training data/' + files[0] + '/data_set_IVa_' + files[0] + '_cnt.txt'
markers       = 'Training data/' + files[0] + '/data_set_IVa_' + files[0] + '_mrk.txt'
    
signal_array = np.loadtxt(training_data)

(b, a) = signal.butter(3, np.array([8, 30])/ 100.00, 'bandpass')
signal_array = signal.lfilter(b, a,signal_array, 1)
            
# #signal_array = np.transpose(signal_array)
    
# marker_array = [map(int, l.split()) for l in open(markers).readlines() ]
        
time = np.arange(len(signal_array))

markers1 = np.loadtxt(markers, dtype = int)
    
train_markers1 = [(float(events[0]),str(events[1])) for events in markers1 if events[1]!= 0]


for events in markers1:
    if events[1] != 0:
        train_markers1.append((float(events[0]) + 100.0, str(events[1])))
        train_markers1.append((float(events[0]) + 200.0, str(events[1])))
    

markers1 = np.array(train_markers1)

y1 = markers1[:, 1]

y1 = [int(value) for value in y1]


# y1 = [0] * len(markers1)

markers_subject1_class_1 = [(float(events[0]), str(events[1])) for events in markers1 if events[1] == '1']
markers_subject1_class_2 = [(float(events[0]), str(events[1])) for events in markers1 if events[1] == '2']

cnt1 = convert_mushu_data(signal_array, markers_subject1_class_1, 100, channels)
cnt2 = convert_mushu_data(signal_array, markers_subject1_class_2, 100, channels)
    
md = {'class 1': ['1'], 'class 2': ['2']}
    
epoch_subject1_class1 = segment_dat(cnt1, md, [0, 1000])
epoch_subject1_class2 = segment_dat(cnt2, md, [0, 1000])

In [5]:
#for 2nd subject

training_data = 'Training data/' + files[1] + '/data_set_IVa_' + files[1] + '_cnt.txt'
markers       = 'Training data/' + files[1] + '/data_set_IVa_' + files[1] + '_mrk.txt'
    
signal_array = np.loadtxt(training_data)

(b, a) = signal.butter(3, np.array([8, 30])/ 100.00, 'bandpass')
signal_array = signal.lfilter(b, a,signal_array, 1)
            
#signal_array = np.transpose(signal_array)
    
# marker_array = [map(int,l.split()) for l in open(markers).readlines()]
        
time = np.arange(len(signal_array))

markers2 = np.loadtxt(markers, dtype = int)
train_markers2 = [(float(events[0]),str(events[1])) for events in markers2 if events[1]!= 0]


for events in markers2:
    if events[1] != 0:
        train_markers2.append((float(events[0]) + 100.0,str(events[1])))
        train_markers2.append((float(events[0]) + 200.0,str(events[1])))
 
markers2 = np.array(train_markers2)
y2 = markers2[:, 1]

y2 = [int(value) for value in y2]


# y2 = [0] * len(markers2)

markers_subject2_class_1 = [(float(events[0]),str(events[1])) for events in markers2 if events[1]== '1']
markers_subject2_class_2 = [(float(events[0]),str(events[1])) for events in markers2 if events[1]== '2']
    
cnt1 = convert_mushu_data(signal_array, markers_subject2_class_1,100,channels)
cnt2 = convert_mushu_data(signal_array, markers_subject2_class_2,100,channels)


    
md = {'class 1': ['1'],'class 2': ['2']}
    
epoch_subject2_class1 = segment_dat(cnt1, md, [0, 1000])
epoch_subject2_class2 = segment_dat(cnt2, md, [0, 1000])



In [6]:
#for 3rd subject

training_data = 'Training data/' + files[2] + '/data_set_IVa_' + files[2] + '_cnt.txt'
markers       = 'Training data/' + files[2] + '/data_set_IVa_' + files[2] + '_mrk.txt'
    
signal_array = np.loadtxt(training_data)

(b, a) = signal.butter(3, np.array([8, 30])/ 100.00, 'bandpass')
signal_array = signal.lfilter(b, a,signal_array, 1)
            
#signal_array = np.transpose(signal_array)
    
# marker_array = [map(int,l.split()) for l in open(markers).readlines()]
        
time = np.arange(len(signal_array))
markers3 = np.loadtxt(markers, dtype = int)
train_markers3 = [(float(events[0]),str(events[1])) for events in markers3 if events[1]!= 0]


for events in markers3:
    if events[1] != 0:
        train_markers3.append((float(events[0]) + 100.0,str(events[1])))
        train_markers3.append((float(events[0]) + 200.0,str(events[1])))
        
markers3 = np.array(train_markers3)
y3 = markers3[:, 1]

y3 = [int(value) for value in y3]


# y3 = [0] * len(markers3)

markers_subject3_class_1 = [(float(events[0]),str(events[1])) for events in markers3 if events[1]== '1']
markers_subject3_class_2 = [(float(events[0]),str(events[1])) for events in markers3 if events[1]== '2']

    
cnt1 = convert_mushu_data(signal_array, markers_subject3_class_1,100,channels)
cnt2 = convert_mushu_data(signal_array, markers_subject3_class_2,100,channels)
    
md = {'class 1': ['1'],'class 2': ['2']}
    
epoch_subject3_class1 = segment_dat(cnt1, md, [0, 1000])
epoch_subject3_class2 = segment_dat(cnt2, md, [0, 1000])

In [7]:
#for 4th subject

training_data = 'Training data/' + files[3] + '/data_set_IVa_' + files[3] + '_cnt.txt'
markers       = 'Training data/' + files[3] + '/data_set_IVa_' + files[3] + '_mrk.txt'
    
signal_array = np.loadtxt(training_data)

(b, a) = signal.butter(3, np.array([8, 30])/ 100.00, 'bandpass')
signal_array = signal.lfilter(b, a,signal_array, 1)
            
#signal_array = np.transpose(signal_array)
    
# marker_array = [map(int,l.split()) for l in open(markers).readlines()]
        
time = np.arange(len(signal_array))
markers4 = np.loadtxt(markers, dtype = int)   
train_markers4 = [(float(events[0]),str(events[1])) for events in markers4 if events[1]!= 0]


for events in markers4:
    if events[1] != 0:
        train_markers4.append((float(events[0])+100.0,str(events[1])))
        train_markers4.append((float(events[0])+200.0,str(events[1])))
        
markers4 = np.array(train_markers4)
y4 = markers4[:, 1]

y4 = [int(value) for value in y4]


# y4 = [0] * len(markers4)


markers_subject4_class_1 = [(float(events[0]),str(events[1])) for events in markers4 if events[1]== '1']
markers_subject4_class_2 = [(float(events[0]),str(events[1])) for events in markers4 if events[1]== '2']
    
    
cnt1 = convert_mushu_data(signal_array, markers_subject4_class_1, 100, channels)
cnt2 = convert_mushu_data(signal_array, markers_subject4_class_2, 100, channels)
    
md = {'class 1': ['1'], 'class 2': ['2']}
    
epoch_subject4_class1 = segment_dat(cnt1, md, [0, 1000])
epoch_subject4_class2 = segment_dat(cnt2, md, [0, 1000])

In [8]:
#for 5th subject

training_data = 'Training data/' + files[4] + '/data_set_IVa_' + files[4] + '_cnt.txt'
markers       = 'Training data/' + files[4] + '/data_set_IVa_' + files[4] + '_mrk.txt'
    
signal_array = np.loadtxt(training_data)

(b, a) = signal.butter(3, np.array([8, 30])/ 100.00, 'bandpass')
signal_array = signal.lfilter(b, a,signal_array, 1)
            
#signal_array = np.transpose(signal_array)
    
# marker_array = [map(int,l.split()) for l in open(markers).readlines()]
        
time = np.arange(len(signal_array))
markers5 = np.loadtxt(markers, dtype = int)      
train_markers5 = [(float(events[0]),str(events[1])) for events in markers5 if events[1]!= 0]


for events in markers5:
    if events[1] != 0:
        train_markers5.append((float(events[0]) + 100.0, str(events[1])))
        train_markers5.append((float(events[0]) + 200.0, str(events[1])))
        
markers5 = np.array(train_markers5)
y5 = markers5[:, 1]

y5 = [int(value) for value in y5]

# y5 = [0] * len(markers5)

markers_subject5_class_1 = [(float(events[0]),str(events[1])) for events in markers5 if events[1]== '1']
markers_subject5_class_2 = [(float(events[0]),str(events[1])) for events in markers5 if events[1]== '2']

    
    
cnt1 = convert_mushu_data(signal_array, markers_subject5_class_1,100,channels)
cnt2 = convert_mushu_data(signal_array, markers_subject5_class_2,100,channels)
    
md = {'class 1': ['1'],'class 2': ['2']}
    
epoch_subject5_class1 = segment_dat(cnt1, md, [0, 1000])
epoch_subject5_class2 = segment_dat(cnt2, md, [0, 1000])

In [9]:
# Appending all the different subject training epochs

temp1_epo1 = append_epo(epoch_subject1_class1, epoch_subject2_class1)
temp1_epo2 = append_epo(epoch_subject3_class1, epoch_subject4_class1)
temp1_epo3 = append_epo(temp1_epo1, temp1_epo2)
class1_epochs = append_epo(temp1_epo3, epoch_subject5_class1)

temp2_epo1 = append_epo(epoch_subject1_class2, epoch_subject2_class2)
temp2_epo2 = append_epo(epoch_subject3_class2, epoch_subject4_class2)
temp2_epo3 = append_epo(temp2_epo1, temp2_epo2)
class2_epochs = append_epo(temp2_epo3, epoch_subject5_class2)


final_epoch = append_epo(class1_epochs, class2_epochs)
s = final_epoch.data[1]
s

array([[ -2.28032167,  -8.74874224, -16.39737541, ..., -28.13124447,
        -28.14914055, -14.52003294],
       [ -1.9854922 , -10.36341916, -25.7352505 , ..., -25.8881739 ,
        -24.93362325, -11.5020445 ],
       [ -2.05459286,  -7.63703252, -13.92491319, ..., -29.08261514,
        -26.63405158, -13.4150343 ],
       ..., 
       [ -1.27605879,  -6.42920443, -16.26957697, ..., -23.5003638 ,
        -20.99038285,  -9.25479407],
       [ -1.41886682,  -4.2613752 ,  -6.1449352 , ..., -22.45830974,
        -20.35207302,  -9.54002527],
       [ -1.15167761,  -5.97264674, -15.5685926 , ..., -22.14528795,
        -20.43684071,  -9.27009135]])

In [10]:
####### FUNCTION TO FIND BAND POWERS ###########

def bandpowers(segment):
    features = []
    
    for i in range(len(segment)):
        f,Psd = signal.welch(segment[i,:], 100)
        power1 = 0
        power2 = 0
        f1 = []
        
        for j in range(0,len(f)):
            if(f[j] >= 4 and f[j] <= 13):
                power1 += Psd[j]
            if(f[j] >= 14 and f[j] <= 30):
                power2 += Psd[j]
        features.append(power1)
        features.append(power2)
    return features

In [11]:
######### Discrete Cosine Transform #########
from scipy.fftpack import fft, dct

def dct_features(segment):
    features = []
    
    for i in range(len(segment)):
        dct_coef = dct(segment[i,:], 2, norm='ortho')
        power = sum( j*j for j in dct_coef)
        features.append(power)
        
    return features

In [12]:
####### FUNCTION TO FIN WAVELET FEATURES #######

def wavelet_features(epoch):
    cA_values = []
    cD_values = []
    cA_mean = []
    cA_std = []
    cA_Energy =[]
    cD_mean = []
    cD_std = []
    cD_Energy = []
    Entropy_D = []
    Entropy_A = []
    features = []
    for i in range(len(epoch)):
        cA,cD=pywt.dwt(epoch[i,:],'coif1')
        cA_values.append(cA)
        cD_values.append(cD)		#calculating the coefficients of wavelet transform.
    for x in range(len(epoch)):   
        cA_Energy.append(abs(np.sum(np.square(cA_values[x]))))
        features.append(abs(np.sum(np.square(cA_values[x]))))
        
    for x in range(len(epoch)):      
        cD_Energy.append(abs(np.sum(np.square(cD_values[x]))))
        features.append(abs(np.sum(np.square(cD_values[x]))))
        
    return features

In [13]:
## FULL FEATURES DICTIONARY

# Dictionary with band power features
#form the dictionary using bandpower features where each column is a data point.

#For class 1
dictionary1 = []
dictionary2 = []
for i in range(len(final_epoch.axes[0])):
    segment = final_epoch.data[i]
    segment = np.array(segment)
    segment = np.transpose(segment)
    
    #features = dct_features(segment)
    features1 = bandpowers(segment)  ## Comment the dictionaries not used
    #features = logvariance(segment)
    
    features2 = wavelet_features(segment)
    
    dictionary1.append(features1)
    dictionary2.append(features2)
    
    
dictionary1 = np.array(dictionary1)
dictionary2 = np.array(dictionary2)


dictionary_bandpower  = dictionary1
dictionary_wavelet = dictionary2

#dictionary_dct = dictionary1


targets = final_epoch.axes[0]

# dictionary1_rh = dictionary1[:,:118]
# # dictionary1_rl = dictionary1.iloc[:,-118:]

# dictionary1_rh
#dictionary1.shape

# dictionary1[0], dictionary1[118]

C:\Users\BCI-HCI-LAB-1\Anaconda2\lib\site-packages\scipy\signal\spectral.py:1637: UserWarning: nperseg = 256 is greater than input length  = 100, using nperseg = 100
  .format(nperseg, input_length))


In [14]:
y_target = y1 + y2 + y3 + y4 + y5
len(y_target)

1680

In [15]:
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(dictionary_bandpower, y_target, test_size=0.1, random_state = 40)

modelSVM = svm.SVC()
modelSVM.fit(X_train,y_train)

y_test_pred = modelSVM.predict(X_test)

metrics.accuracy_score(y_test, y_test_pred, normalize= True, sample_weight=None)

0.65476190476190477

In [16]:
s_transpose = np.transpose(s)
feature_s = bandpowers(s_transpose)
len(feature_s)

236

### APPLYING ML ALGORITHMS FOR BAND POWER FEATURES

### SUPPORT VECTOR MACHINES

In [17]:
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(dictionary_bandpower, y_target, test_size=0.1, random_state = 40)

modelSVM = svm.SVC()
modelSVM.fit(X_train,y_train)

y_test_pred = modelSVM.predict(X_test)

metrics.accuracy_score(y_test, y_test_pred, normalize= True, sample_weight=None)

0.65476190476190477

### GAUSSIAN NAIVE BAYES 

In [18]:
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics


X_train, X_test, y_train, y_test = train_test_split(dictionary_bandpower, y_target, test_size=0.1, random_state=30)

gnb = GaussianNB()

y_pred = gnb.fit(X_train, y_train).predict(X_test)

metrics.accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)

C:\Users\BCI-HCI-LAB-1\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.5

### MULTI LAYER PERCEPTRON CLASSIFIER

In [19]:
from sklearn.neural_network import MLPClassifier

X_train, X_test, y_train, y_test = train_test_split(dictionary_bandpower, y_target, test_size = 0.1, random_state=40)

modelMLP = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes = (5, 2), random_state=1)
 
modelMLP.fit(X_train, y_train)

y_test_pred = modelMLP.predict(X_test)

metrics.accuracy_score(y_test, y_test_pred, normalize=True, sample_weight=None)

0.47619047619047616

### ADA BOOST CLASSIFER

In [20]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

modelAda = AdaBoostClassifier(n_estimators = 200)
scores = cross_val_score(modelAda, dictionary_bandpower, y_target)
scores.mean()     

0.50892857142857151

### LINEAR DISCRIMINANT ANALYSIS

In [21]:
from sklearn.lda import LDA

X_train, X_test, y_train, y_test = train_test_split(dictionary_bandpower, y_target, test_size=0.1, random_state=40)

modelLDA = LDA()

modelLDA.fit(X_train, y_train)

y_test_pred = modelLDA.predict(X_test)

metrics.accuracy_score(y_test, y_test_pred, normalize=True, sample_weight=None)

C:\Users\BCI-HCI-LAB-1\Anaconda2\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


0.59523809523809523

### GRADIENT BOOSTING CLASSIFIER

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(dictionary_bandpower, y_target, test_size=0.1, random_state=40)

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test) 

0.52976190476190477

### DECISION TREE CLASSIFIER

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_gaussian_quantiles

X_train, X_test, y_train, y_test = train_test_split(dictionary_bandpower, y_target, test_size=0.1, random_state=40)

bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                         algorithm="SAMME",
                         n_estimators = 200)

bdt.fit(X_train, y_train)

y_test_pred = bdt.predict(X_test)

metrics.accuracy_score(y_test, y_test_pred, normalize=True, sample_weight=None)


0.5535714285714286

### RANDOM FOREST CLASSIFIER

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


modelRF = RandomForestClassifier(n_estimators = 5, max_depth=None, min_samples_split=3, random_state=10)
scores = cross_val_score(modelRF, dictionary_bandpower, y_target)
scores.mean()         

0.51726190476190481

### Nu SUPPORT VECTOR CLASSIFIER

In [25]:
X_train, X_test, y_train, y_test = train_test_split(dictionary_bandpower, y_target, test_size=0.1, random_state=40)
modelSVM = svm.NuSVC()
modelSVM.fit(X_train,y_train)

y_test_pred = modelSVM.predict(X_test)

metrics.accuracy_score(y_test, y_test_pred, normalize=True, sample_weight=None)

0.63690476190476186

### DECISION TREE CLASSIFIER

In [26]:
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(dictionary_bandpower, y_target, test_size=0.1, random_state=40)

modelDecisionTree = tree.DecisionTreeClassifier()
modelDecisionTree = modelDecisionTree.fit(X_train, y_train)


y_test_pred = modelDecisionTree.predict(X_test)

metrics.accuracy_score(y_test, y_test_pred, normalize=True, sample_weight=None)

0.5714285714285714

### CONTINUED IN THE NEXT NOTEBOOK

### USING ONLY 30 CHANNELS PRESENT ON THE MOTOR CORTEX

In [27]:
channels, len(channels)

(['Fp1',
  'AFp1',
  'Fpz',
  'AFp2',
  'Fp2',
  'AF7',
  'AF3',
  'AF4',
  'AF8',
  'FAF5',
  'FAF1',
  'FAF2',
  'FAF6',
  'F7',
  'F5',
  'F3',
  'F1',
  'Fz',
  'F2',
  'F4',
  'F6',
  'F8',
  'FFC7',
  'FFC5',
  'FFC3',
  'FFC1',
  'FFC2',
  'FFC4',
  'FFC6',
  'FFC8',
  'FT9',
  'FT7',
  'FC5',
  'FC3',
  'FC1',
  'FCz',
  'FC2',
  'FC4',
  'FC6',
  'FT8',
  'FT10',
  'CFC7',
  'CFC5',
  'CFC3',
  'CFC1',
  'CFC2',
  'CFC4',
  'CFC6',
  'CFC8',
  'T7',
  'C5',
  'C3',
  'C1',
  'Cz',
  'C2',
  'C4',
  'C6',
  'T8',
  'CCP7',
  'CCP5',
  'CCP3',
  'CCP1',
  'CCP2',
  'CCP4',
  'CCP6',
  'CCP8',
  'TP9',
  'TP7',
  'CP5',
  'CP3',
  'CP1',
  'CPz',
  'CP2',
  'CP4',
  'CP6',
  'TP8',
  'TP10',
  'PCP7',
  'PCP5',
  'PCP3',
  'PCP1',
  'PCP2',
  'PCP4',
  'PCP6',
  'PCP8',
  'P9',
  'P7',
  'P5',
  'P3',
  'P1',
  'Pz',
  'P2',
  'P4',
  'P6',
  'P8',
  'P10',
  'PPO7',
  'PPO5',
  'PPO1',
  'PPO2',
  'PPO6',
  'PPO8',
  'PO7',
  'PO3',
  'PO1',
  'POz',
  'PO2',
  'PO4',
  'PO8',
 

In [28]:
channels_motor = [ 'FC2', 'FC4', 'FC6', 'CFC2', 'CFC4', 'CFC6', 'C2', 'C4', 'C6', 'CCP2', 'CCP4', 'CCP6', 'CP2', 
                  'CP4', 'CP6', 'FC5', 'FC3', 'FC1', 'CFC5', 'CFC3', 'CFC1', 'C5', 'C3', 'C1', 'CCP5', 'CCP3', 'CCP1', 
                  'CP5', 'CP3', 'CP1']
len(channels_motor)

30

In [29]:
signal_array.shape

(283562L, 118L)

In [67]:
indices = []

for i, j in enumerate(channels):
    for k in channels_motor:
        if j == k:
            indices.append(i)
            
indices

[32,
 33,
 34,
 36,
 37,
 38,
 42,
 43,
 44,
 45,
 46,
 47,
 50,
 51,
 52,
 54,
 55,
 56,
 59,
 60,
 61,
 62,
 63,
 64,
 68,
 69,
 70,
 72,
 73,
 74]

In [77]:
import pandas as pd

data = pd.read_csv('training_data', header = None, names = channels)
data.head()

IOError: File training_data does not exist